In [1]:
import pandas as pd
import numpy as np

In [2]:
Years = range(1990, 2018)
#year = 2015

In [3]:
#Umpire id, Umpire Name, AB, K , Walk, 
#Catcher id, Catcher Name, Steal Attempts,Caught Stealing, Count

In [4]:
First = True
for year in Years:
    rLog = pd.read_csv("./RetrosheetData/GameLogs/gl"+str(year)+".txt", header=None)
    for row in rLog.values:
        #Get Umpire id
        Row = rLog.values[0]
        UmpireH = row[77:79]

        #Get the Catcher IDs
        CatcherH=row[132+row[132:159].tolist().index(2)-2:132+row[132:159].tolist().index(2)]
        CatcherA=row[105+row[105:132].tolist().index(2)-2:105+row[105:132].tolist().index(2)]
        
        #Park ID
        ParkID = row[16]
        
        #Home Team
        HomeTeamOffensiveStats = [float(x) for x in row[49:66]]
        ABH = HomeTeamOffensiveStats[0]
        KH = HomeTeamOffensiveStats[11]
        WH = HomeTeamOffensiveStats[9]
        OnBaseH = sum(HomeTeamOffensiveStats[1:5])
        StealAttemptH = HomeTeamOffensiveStats[12]+HomeTeamOffensiveStats[13]
        CaughtStealingH = HomeTeamOffensiveStats[13]

        #Away Team
        AwayTeamOffensiveStats = [float(x) for x in row[21:38]]
        ABA = AwayTeamOffensiveStats[0]
        KA = AwayTeamOffensiveStats[11]
        WA = AwayTeamOffensiveStats[9]
        OnBaseA = sum(AwayTeamOffensiveStats[1:5])
        StealAttemptA = AwayTeamOffensiveStats[12]+AwayTeamOffensiveStats[13]
        CaughtStealingA = AwayTeamOffensiveStats[13]

        #Build Ump Row
        URow = np.append(UmpireH, [ABH+ABA, KH+KA, WH+WA,OnBaseH+OnBaseA])

        #Build Catcher Rows
        CHRow = np.append(CatcherH,[float(StealAttemptA), float(CaughtStealingA), 1])
        CARow = np.append(CatcherA, [float(StealAttemptH), float(CaughtStealingH), 1])
        
        #Build Park Rows
        ParkRow = np.array([ParkID]+[sum(x) for x in zip(HomeTeamOffensiveStats[0:6],AwayTeamOffensiveStats[0:6])], dtype=object)
        
        #Check if ID is already in Data
        if First:
            #Add the first row in 
            UmpireData = np.array([URow], dtype=object)
            CatcherData = np.array([CHRow,CARow], dtype=object)
            ParkData = np.array([ParkRow], dtype=object)
            First=False

        #If IDs are  in the data aggregate stats.
        else:
            #Umpire data
            if URow[0] in UmpireData[:,0]:
                #Get the index:
                indexu=np.where(UmpireData[:,0]==URow[0])[0][0]
                UmpireData[indexu,2:6]=UmpireData[indexu,2:6]+URow[2:6]
            else:
                UmpireData=np.append(UmpireData,[URow],axis=0)

            #Home Catcher
            if CHRow[0] in CatcherData[:,0]:
                indexch = np.where(CatcherData[:,0]==CHRow[0])[0][0]
                CatcherData[indexch,2:5] = CatcherData[indexch,2:5] + CHRow[2:5]
            else:
                CatcherData=np.append(CatcherData,[CHRow],axis=0)
            #Away Catcher
            if CARow[0] in CatcherData[:,0]:
                indexca = np.where(CatcherData==CARow[0])[0][0]
                CatcherData[indexca,2:5] = CatcherData[indexca,2:5] + CARow[2:5]
            else:
                CatcherData=np.append(CatcherData,[CARow],axis=0)   
                
            if ParkRow[0] in ParkData[:,0]:
                indexp=np.where(ParkData[:,0]==ParkRow[0])[0][0]
                ParkData[indexp,1:7] = ParkData[indexp,1:7] + ParkRow[1:7]
            else:
                ParkData=np.append(ParkData,[ParkRow],axis=0) 
                
                

In [5]:
#Create Pandas DataFrame
UmpireStats = pd.DataFrame(data=UmpireData, columns=['Umpire Id', 'Umpire Name', 'AB', 'K' , 'Walk', '1H/2H/3H/HR'])
CatcherStats = pd.DataFrame(data=CatcherData, columns=['Catcher Id', 'Catcher Name', 'Steal Attempts','Caught Stealing', 'Count'])
ParkStats = pd.DataFrame(data=ParkData, columns=['Park ID','AB','H','2H','3H', 'HR', 'RBI'])

In [6]:
#Calculate Percentages
UmpireStats['K%'] = UmpireStats['K']/UmpireStats['AB']*100
UmpireStats['Walk%'] = UmpireStats['Walk']/UmpireStats['AB']*100
UmpireStats['Hit%'] = UmpireStats['1H/2H/3H/HR']/UmpireStats['AB']*100

#Calulate the Means
KpMean = np.mean(UmpireStats['K%'].values)
WalkpMean = np.mean(UmpireStats['Walk%'].values)
HitpMean = np.mean(UmpireStats['Hit%'].values)

#Calculate the factors.
UmpireStats['K Factor'] = UmpireStats['K%']/KpMean
UmpireStats['Walk Factor'] = UmpireStats['Walk%']/WalkpMean
UmpireStats['Hit Factor'] = UmpireStats['Hit%']/HitpMean

#Check the Header
UmpireStats.head()

,Umpire Id,Umpire Name,AB,K,Walk,1H/2H/3H/HR,K%,Walk%,Hit%,K Factor,Walk Factor,Hit Factor
0,barnl901,Larry Barnett,20290,3517,2036,7218,17.3337,10.0345,35.5742,0.882563,0.996827,1.03816
1,mckej901,Jim McKean,22459,3905,2164,8042,17.3872,9.63534,35.8075,0.885291,0.957174,1.04497
2,garcr901,Rich Garcia,22230,3684,2289,7950,16.5722,10.2969,35.7625,0.843792,1.02289,1.04366
3,denkd901,Don Denkinger,15092,2408,1617,5226,15.9555,10.7143,34.6276,0.812391,1.06436,1.01054
4,brinj901,Joe Brinkman,35308,6272,3579,12462,17.7637,10.1365,35.2951,0.904458,1.00696,1.03002


In [7]:
CatcherStats.head()

,Catcher Id,Catcher Name,Steal Attempts,Caught Stealing,Count
0,penat001,Tony Pena,674,213,675
1,nokem001,Matt Nokes,435,123,339
2,parrl001,Lance Parrish,404,160,362
3,valld001,Dave Valle,508,180,529
4,fiskc001,Carlton Fisk,285,102,267


In [8]:
#Take out the Zero Factors
CatcherStatsZero=CatcherStats[CatcherStats['Steal Attempts']==0]
CatcherStatsNonZero=CatcherStats[CatcherStats['Steal Attempts']!=0]

#Calcuate the Factors
CatcherStatsNonZero['Steals Attempts per Game'] = CatcherStatsNonZero['Steal Attempts']/CatcherStatsNonZero['Count']
CatcherStatsNonZero['Catcher Success'] = CatcherStatsNonZero['Caught Stealing']/CatcherStatsNonZero['Steal Attempts']

#CatcherStatsZero['Steals Attempts per Game'] = np.zeros(CatcherStatsZero.size)
#CatcherStatsZero['Catcher Success'] = np.zeros(CatcherStatsZero.size)

CatcherStats = pd.concat([CatcherStatsNonZero,CatcherStatsZero],ignore_index=True)
CatcherStats.head()

/home/josh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/josh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


,Catcher Id,Catcher Name,Catcher Success,Caught Stealing,Count,Steal Attempts,Steals Attempts per Game
0,penat001,Tony Pena,0.316024,213,675,674,0.998519
1,nokem001,Matt Nokes,0.282759,123,339,435,1.28319
2,parrl001,Lance Parrish,0.39604,160,362,404,1.11602
3,valld001,Dave Valle,0.354331,180,529,508,0.960302
4,fiskc001,Carlton Fisk,0.357895,102,267,285,1.06742


In [9]:
ParkStats.head()

,Park ID,AB,H,2H,3H,HR,RBI
0,BOS07,155627,42655,9694,775,4374,21242
1,ANA01,153678,40247,7404,578,4540,19169
2,CHI10,5360,1358,224,53,94,610
3,KAN06,154771,42262,8321,1142,3929,20318
4,OAK01,151020,38144,7321,664,4345,18808


In [10]:
ParkStats['Hits Per AB'] = ParkStats['H']/ParkStats['AB']
ParkStats['Doubles Per AB'] = ParkStats['2H']/ParkStats['AB']
ParkStats['Triples Per AB'] = ParkStats['3H']/ParkStats['AB']
ParkStats['HR Per AB'] = ParkStats['HR']/ParkStats['AB']
ParkStats['Runs Per AB'] = ParkStats['RBI']/ParkStats['AB']

HitMean = np.mean(ParkStats['Hits Per AB'])
DoubleMean = np.mean(ParkStats['Doubles Per AB'])
TripleMean = np.mean(ParkStats['Triples Per AB'])
HRMean = np.mean(ParkStats['HR Per AB'])
RunMean = np.mean(ParkStats['Runs Per AB'])

ParkStats['Hits Factor'] = ParkStats['Hits Per AB']/HitMean
ParkStats['Double Factor'] = ParkStats['Doubles Per AB']/DoubleMean
ParkStats['Triple Factor'] = ParkStats['Triples Per AB']/TripleMean
ParkStats['HR Factor'] = ParkStats['HR Per AB']/HRMean
ParkStats['RBI Factor'] = ParkStats['Runs Per AB']/RunMean

In [11]:
ParkStats.head()

,Park ID,AB,H,2H,3H,HR,RBI,Hits Per AB,Doubles Per AB,Triples Per AB,HR Per AB,Runs Per AB,Hits Factor,Double Factor,Triple Factor,HR Factor,RBI Factor
0,BOS07,155627,42655,9694,775,4374,21242,0.274085,0.06229,0.00497986,0.0281057,0.136493,1.04932,1.25528,0.824889,0.975609,1.07841
1,ANA01,153678,40247,7404,578,4540,19169,0.261892,0.0481787,0.00376111,0.0295423,0.124735,1.00263,0.970902,0.62301,1.02548,0.985506
2,CHI10,5360,1358,224,53,94,610,0.253358,0.041791,0.00988806,0.0175373,0.113806,0.969965,0.842178,1.63791,0.608758,0.899159
3,KAN06,154771,42262,8321,1142,3929,20318,0.273061,0.0537633,0.00737864,0.0253859,0.131278,1.0454,1.08344,1.22224,0.881199,1.0372
4,OAK01,151020,38144,7321,664,4345,18808,0.252576,0.048477,0.00439677,0.028771,0.12454,0.96697,0.976915,0.728304,0.998705,0.983965


In [12]:
#Output the csv files.
UmpireStats.to_csv("./Factors/UmpireFactors.csv")
CatcherStats.to_csv("./Factors/CatcherFactors.csv")
ParkStats.to_csv("./Factors/ParkFactors.csv")